In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import cmocean
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
matplotlib.rc_file('../rc_file_paper')
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from scipy.optimize import curve_fit
from paths import path_samoc, path_results
from paths import file_ex_ocn_ctrl, file_ex_ocn_lpd
from xr_regression import xr_quadtrend
from bb_analysis_timeseries import AnalyzeTimeSeries as ATS

color conventions:
- global black
- Atlantic blue
- Pacific orange
- Southern red

In [ ]:
# OHC data
ctrl = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl.nc')
ctrl = ctrl.assign_coords(time=ctrl.time.values/365)
lpd  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_lpd.nc' )
lpd  = lpd.assign_coords(time=lpd .time.values/365)

ctrl_qd = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_qd.nc', decode_times=False)
lpd_qd  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_lpd_qd.nc' , decode_times=False)

In [ ]:
# SST data
SST_ctrl = xr.open_dataset(f'{path_samoc}/SST/SST_mean_timeseries_ctrl.nc')
SST_lpd  = xr.open_dataset(f'{path_samoc}/SST/SST_mean_timeseries_lpd.nc')
SST_ctrl = SST_ctrl.assign_coords(time=SST_ctrl.time.values/365)
SST_lpd  = SST_lpd.assign_coords(time=SST_lpd.time.values/365)

In [ ]:
spinup = 50  # how many years to ignore due to spinup effects: data from year 51 of ctrl run

ctrl = ctrl.isel(time=np.arange(50,300))
lpd  = lpd .isel(time=np.arange(0,300-spinup))

# Fig 1: exp decay $\Delta$OHC / SST
data from `SST.ipynb`

plot from `OHC-ctrl_vs_lpd.ipynb`

In [ ]:
f, ax = plt.subplots(1, 2, sharey=True, figsize=(6.4, 2.5))
for i, da in enumerate([ctrl, lpd]):
    ax[i].axhline(0, c='grey', lw=.5)
    ax[i].plot(da.time, (da['OHC_Global_Ocean']-da['OHC_Global_Ocean'].shift(time=1))/1e21, c='k')
    ax[i].set_xlabel('time [model years]')
    ax[i].text(.05,.9, ['HIGH', 'LOW'][i], transform=ax[i].transAxes)
Delta = r'$\Delta$'
ax[0].set_ylabel(f'annual difference {Delta}OHC [ZJ]')
plt.savefig(f'{path_results}/paper/Delta_OHC_ctrl_lpd')

## Table 2: $\Delta$OHC mean and std

In [ ]:
for i, ocean in enumerate(['Global', 'Atlantic', 'Pacific', 'Southern']):
    means, stds = [], []
    for j, da in enumerate([ctrl, lpd]):
        da_ = (da[f'OHC_{ocean}_Ocean']-da[f'OHC_{ocean}_Ocean'].shift(time=1)).sel(time=slice(200,300))/1e21
#         print(da_.time.sel(time=slice(200,300)))
        means.append(da_.mean().values)
        stds.append(da_.std().values)
    print(f'{ocean:8} & ${means[0]:4.1f} \pm {stds[0]:3.1f}$ & ${means[1]:3.1f} \pm {stds[1]:3.1f}$ \\\\')

# Fig 2: SST index regression patterns
from `OHC-ctrl_vs_lpd.ipynb`

make orthographic AMO maps as well

# Fig 3: SST index spectra
from `SST_indices.ipynb`

! recompute

In [ ]:
f, ax = plt.subplots(3, 3, figsize=(6.4,4), sharex=True, constrained_layout=True)
for i, run in enumerate(['had', 'ctrl', 'lpd']):
    if run=='had':
        dt = 'GMST_tfdt'
        tslice = slice(149)
    elif run=='ctrl':
        dt = 'quadratic_pwdt'
        tslice = slice(1,251)  # ctrl AMO time series starts in year 50
    elif run=='lpd':
        dt = 'quadratic_pwdt'
        tslice = slice(250)
        
    for j, idx in enumerate(['AMO', 'SOM', 'TPI']):
        if idx=='TPI':
            da1 = xr.open_dataarray(f'{path_samoc}/SST/{idx}1_{dt}_raw_{run}.nc')
            da2 = xr.open_dataarray(f'{path_samoc}/SST/{idx}2_{dt}_raw_{run}.nc')
            da3 = xr.open_dataarray(f'{path_samoc}/SST/{idx}3_{dt}_raw_{run}.nc')
            da = da2 - (da1+da3)/2
        else:
            da = xr.open_dataarray(f'{path_samoc}/SST/{idx}_{dt}_raw_{run}.nc')
        
        da_select = da.isel(time=tslice)
        assert len(da_select) in [149, 250]
        
        ft, fc = 'lowpass', 13
        spec = ATS(da_select).spectrum(filter_type=ft, filter_cutoff=fc)  # spectrum
        rnsp = ATS(da_select).mc_ar1_spectrum(filter_type=ft, filter_cutoff=fc)  # red noise spectrum

#         ax[j,i].plot(1/rnsp[1,:], rnsp[3,:]   , c='C1',                 label='AR(1) 95% C.I.')
        ax[j,i].loglog(1/rnsp[1,:], rnsp[3,:]   , c='C1',                 label='AR(1) 95% C.I.')
#         ax[j,i].plot(1/rnsp[1,:], rnsp[0,:]   , c='C1', lw=.5, ls=':' , label='AR(1) median')
#         ax[j,i].plot(1/spec[1]  , spec[0]     , c='C0',                 label='MT spectrum')
        ax[j,i].loglog(1/spec[1]  , spec[0]     , c='C0',                 label='MT spectrum')
#         ax[j,i].loglog(1/spec[1], spec[2].T[1], c='C0', lw=.5, ls='--', label='jackknife est.')
        ax[j,i].set_xlim((10,100))
        ax[j,i].set_ylim((1e-4, 5*np.max(rnsp[3,:].T[1])))
        ax[j,i].set_yticklabels([])
        ax[j,i].xaxis.set_minor_formatter(NullFormatter())
        ax[j,0].set_ylabel(f'{idx} index\nspectral power')
        if i==0 and j==0:  ax[j,i].legend(loc=4, fontsize=8, frameon=False)
#         ax[j,i].set_xticklabels([])
        ax[j,i].get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
    ax[2,i].set_xticks([10,20,40,80])
    ax[2,i].set_xticklabels([10,20,40,80])
    
    ax[0,i].title.set_text(['HIST', 'HIGH', 'LOW'][i])
    ax[2,i].set_xlabel('period [years]')
plt.savefig(f'{path_results}/paper/spectra')

# Fig 4: OHC time series

In [ ]:
from filters import lowpass

In [ ]:
f, ax = plt.subplots(1, 2 , figsize=(6.4,2.5), sharey=True)
for i in range(2):
    ax[i].axhline(0, c='grey', lw=.5)
    ax[i].set_xlabel('time [model years]')

for i, ocean in enumerate(['Global', 'Atlantic', 'Pacific', 'Southern']):
    key = f'OHC_{ocean}_Ocean'
    c = ['k' ,'C0','C1','C3'][i]
    for j, da in enumerate([ctrl_qd, lpd_qd]):
        x = da[key]/1e21
#         ax[j].plot(x.time, x, lw=.5, c=c ,label=f'{ocean}')
        ax[j].plot(x.time, lowpass(x,5), c=c ,label=f'{ocean}')
    
for j in range(2):
    ax[j].text(.05,.9, ['HIGH', 'LOW'][j], transform=ax[j].transAxes)
ax[0].set_ylabel('OHC anomaly [ZJ]')
ax[1].legend(fontsize=8, ncol=2, frameon=False)
plt.savefig(f'{path_results}/paper/OHC_time_series')

# Fig 5: OHC depth-zonal integral Hovmöller diagram + standard deviation (y)

In [ ]:
lpd_lat = lpd.TLAT.mean(axis=1)
extents = [(-78,90), (-34,70), (-34,70), (-34,30)]
height_ratios = [a[1]-a[0] for a in extents]
f, ax = plt.subplots(4, 4, figsize=(6.4,8), sharex='col',
                     gridspec_kw={"width_ratios":[1, 1, 0.05, .5], "height_ratios":height_ratios, "wspace":0.03, "hspace":0.03})
cY, cX = np.meshgrid(ctrl.t_lat, ctrl.time)
lY, lX = np.meshgrid(lpd.TLAT.mean(axis=1), lpd.time)
vex, ims = [2.5e16, 1.5e16, 2e16, .7e16], []
for i, ocean in enumerate(['Global', 'Atlantic', 'Pacific', 'Indian']):
    kwargs = {'cmap':cmocean.cm.balance, 'vmin':-vex[i], 'vmax':vex[i]}
    key = f'OHC_zonal_{ocean}_Ocean'
    im = ax[i,0].pcolormesh(cX, cY, ctrl[key]-xr_quadtrend(ctrl[key]), **kwargs)
    ims.append(im)
    ax[i,1].pcolormesh(lX, lY, (lpd[key]-xr_quadtrend(lpd[key]))/100, **kwargs)
    for j in [0,1,3]:  
        ax[i,j].axhline(0, c='grey', lw=.5, ls='--')
        ax[i,j].set_yticks(np.arange(-60,100,30))
        ax[i,j].set_ylim(extents[i])
    ax[i,1].set_yticklabels([])
    ax[i,0].text(60, extents[i][1]-10, ocean, c='g')
    ax[i,0].set_ylabel('latitude')
    
    ax[i,3].plot((ctrl[key]-xr_quadtrend(ctrl[key])).std(dim='time')    , ctrl.t_lat, c='k')
    ax[i,3].plot((lpd[key] -xr_quadtrend(lpd[key] )).std(dim='time')/100, lpd_lat   , c='k', ls='--')
    ax[i,3].set_yticklabels([])
    
    ax[i,0].get_shared_y_axes().join(ax[i,0], ax[i,1])
    ax[i,0].get_shared_y_axes().join(ax[i,0], ax[i,3])
    
    cb = f.colorbar(ims[i], cax=ax[i,2])#, ticks=np.arange(-3e16,4e16,1e16))
    cb.outline.set_visible(False)
    
ax[0,0].text(60, -75, 'Southern Ocean', c='g')
for j in range(2):
    ax[0,j].axhline(-31.5, c='g', lw=.8)
    ax[0,j].text(.5, 1.02, ['HIGH', 'LOW'][j], transform=ax[0,j].transAxes, ha='center')
    ax[-1,j].set_xlabel('time [model years]')
    ax[-1,-j-1].set_xlabel('[ZJ/m]')
ax[0,3].text(.5, 1.02, 'st. dev.', transform=ax[0,3].transAxes, ha='center')
    
f.align_xlabels()
plt.savefig(f'{path_results}/paper/OHC_zonal_Hovmoeller')

In [ ]:
lpd_lat = lpd.TLAT.mean(axis=1)
extents = [(-78,90), (-34,70), (-34,70), (-34,30)]
height_ratios = [a[1]-a[0] for a in extents]
f, ax = plt.subplots(4, 4, figsize=(6.4,8), sharex='col',
                     gridspec_kw={"width_ratios":[1, 1, 0.05, .5], "height_ratios":height_ratios, "wspace":0.03, "hspace":0.03})
cY, cX = np.meshgrid(ctrl.t_lat, ctrl.time)
lY, lX = np.meshgrid(lpd.TLAT.mean(axis=1), lpd.time)
vex, ims = [2e16, 1.5e16, 1.5e16, .5e16], []
for i, ocean in enumerate(['Global', 'Atlantic', 'Pacific', 'Indian']):
    kwargs = {'cmap':cmocean.cm.balance, 'vmin':-vex[i], 'vmax':vex[i]}
    key = f'OHC_zonal_{ocean}_Ocean'
    im = ax[i,0].pcolormesh(cX, cY, lowpass(ctrl[key]-xr_quadtrend(ctrl[key]),13), **kwargs)
    ims.append(im)
    ax[i,1].pcolormesh(lX, lY, lowpass((lpd[key]-xr_quadtrend(lpd[key]))/100,13), **kwargs)
    for j in [0,1,3]:  
        ax[i,j].axhline(0, c='grey', lw=.5, ls='--')
        ax[i,j].set_yticks(np.arange(-60,100,30))
        ax[i,j].set_ylim(extents[i])
    ax[i,1].set_yticklabels([])
    ax[i,0].text(60, extents[i][1]-10, ocean, c='g')
    ax[i,0].set_ylabel('latitude')
    
    ax[i,3].plot(lowpass(ctrl[key]-xr_quadtrend(ctrl[key]),13).std(dim='time')    , ctrl.t_lat, c='k')
    ax[i,3].plot(lowpass(lpd[key] -xr_quadtrend(lpd[key] ),13).std(dim='time')/100, lpd_lat   , c='k', ls='--')
    ax[i,3].set_yticklabels([])
    
    ax[i,0].get_shared_y_axes().join(ax[i,0], ax[i,1])
    ax[i,0].get_shared_y_axes().join(ax[i,0], ax[i,3])
    
    cb = f.colorbar(ims[i], cax=ax[i,2])#, ticks=np.arange(-3e16,4e16,1e16))
    cb.outline.set_visible(False)
    
ax[0,0].text(60, -75, 'Southern Ocean', c='g')
for j in range(2):
    ax[0,j].axhline(-31.5, c='g', lw=.8)
    ax[0,j].text(.5, 1.02, ['HIGH', 'LOW'][j], transform=ax[0,j].transAxes, ha='center')
    ax[-1,j].set_xlabel('time [model years]')
    ax[-1,-j-1].set_xlabel('[ZJ/m]')
ax[0,3].text(.5, 1.02, 'st. dev.', transform=ax[0,3].transAxes, ha='center')
    
f.align_xlabels()
plt.savefig(f'{path_results}/paper/OHC_zonal_Hovmoeller_lowpass13')

Gulf Stream Separation point

In [ ]:
da1 = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).TEMP[0,0,:,:]
da2 = xr.open_dataset(file_ex_ocn_lpd , decode_times=False).TEMP[0,0,:,:]

In [ ]:
fig = plt.figure(figsize=(12,7))
for i, run in enumerate(['ctrl', 'lpd']):
    da = [da1, da2][i]
    lats = da.TLAT
    lons = da.TLONG
    
    ax = fig.add_subplot(2, 1, i+1,
                          projection=ccrs.NearsidePerspective(central_longitude=-70, central_latitude=35, satellite_height=357858))
    ax.set_position([[.02,.52][i],.05,.46,.93])
    im = ax.pcolormesh(lons, lats, da.values,
                        cmap=cmocean.cm.balance, vmin=5, vmax=30,
                        transform=ccrs.PlateCarree(),
                        )
        # grid
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
    gl.ylocator = matplotlib.ticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
    gl.xlocator = matplotlib.ticker.FixedLocator([-180, -120, -60, 0, 60, 120, 180])

# Fig 6: OHC horizontal integral Hovmöller diagram

In [ ]:
oceans = ['Global', 'Atlantic', 'Pacific', 'Southern']
das = [ctrl_qd, lpd_qd]
maxv = .06

fig = plt.figure(figsize=(6.4,9), constrained_layout=True)
gs0 = matplotlib.gridspec.GridSpec(4, 2, left=.1, right=.98, bottom=.11, top=.98, wspace=.05, hspace=.045)


# if offset==True: x = (da-da.isel(time=slice(0,30)).mean(dim='time')).T/1e21

for i, ocean in enumerate(oceans):
    name = f'{ocean}_Ocean'    
    
    for j, da in enumerate([da1, da2]):
        da = das[j][f'OHC_levels_{name}']
        x = da.T/1e21
#         X, Y = np.meshgrid(da.time, -da.coords[['depth_t', 'z_t'][j]]/[1, 1e2][j])
        X, Y = np.meshgrid(da.time, da.coords[['depth_t', 'z_t'][j]]/[1, 1e2][j])
        
        gs00 = matplotlib.gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=gs0[i,j], hspace=0)
        ax_top = fig.add_subplot(gs00[0])
        ax_top.set_ylim((-1500,0))
        ax_top.set_xticks([])
        ax_top.set_yticks([-1500, -1000, -500, 0])
        
        ax_bot = fig.add_subplot(gs00[1])
        ax_bot.set_ylim((-6000,-1500))
        ax_bot.set_yticks([-6000,-4500,-3000])
                
        for k, ax in enumerate([ax_top, ax_bot]):
            im = ax.pcolormesh(X, -Y, x, vmin=-maxv, vmax=maxv, cmap=cmocean.cm.balance)
        
        if j==0:
            ax_bot.set_ylabel(f'{ocean} Ocean', horizontalalignment = 'left')
        
        if j==1:
            ax_top.set_yticks([])
            ax_bot.set_yticks([])
                
        if i==0:
            ax_top.text(.5,1.05,['HIGH', 'LOW'][j], transform=ax_top.transAxes)
        if i==len(oceans)-1:
            ax_bot.set_xlabel('time [model years]')
        else:
            ax_bot.set_xticks([])
            
            
cax = fig.add_axes([0.1, 0.04, 0.88, 0.02])
fig.colorbar(im, cax=cax, orientation='horizontal', label='OHC anomaly [ZJ/m]', extend='both')
plt.savefig(f'{path_results}/paper/OHC_vertical_Hovmoeller_0-6km_ctrl_lpd_qd')


In [ ]:
oceans = ['Global', 'Atlantic', 'Pacific', 'Southern']
das = [ctrl_qd, lpd_qd]
maxv = .06

fig = plt.figure(figsize=(6.4,9), constrained_layout=True)
gs0 = matplotlib.gridspec.GridSpec(4, 2, left=.1, right=.98, bottom=.11, top=.98, wspace=.05, hspace=.045)


# if offset==True: x = (da-da.isel(time=slice(0,30)).mean(dim='time')).T/1e21

for i, ocean in enumerate(oceans):
    name = f'{ocean}_Ocean'    
    
    for j, da in enumerate([da1, da2]):
        da = das[j][f'OHC_levels_{name}']
        x = da.T/1e21
#         X, Y = np.meshgrid(da.time, -da.coords[['depth_t', 'z_t'][j]]/[1, 1e2][j])
        X, Y = np.meshgrid(da.time, da.coords[['depth_t', 'z_t'][j]]/[1, 1e2][j])
        
        gs00 = matplotlib.gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=gs0[i,j], hspace=0)
        ax_top = fig.add_subplot(gs00[0])
        ax_top.set_ylim((-1500,0))
        ax_top.set_xticks([])
        ax_top.set_yticks([-1500, -1000, -500, 0])
        
        ax_bot = fig.add_subplot(gs00[1])
        ax_bot.set_ylim((-6000,-1500))
        ax_bot.set_yticks([-6000,-4500,-3000])
                
        for k, ax in enumerate([ax_top, ax_bot]):
            im = ax.pcolormesh(X, -Y, lowpass(x.T,13).T, vmin=-maxv, vmax=maxv, cmap=cmocean.cm.balance)
        
        if j==0:
            ax_bot.set_ylabel(f'{ocean} Ocean', horizontalalignment = 'left')
        
        if j==1:
            ax_top.set_yticks([])
            ax_bot.set_yticks([])
                
        if i==0:
            ax_top.text(.5,1.05,['HIGH', 'LOW'][j], transform=ax_top.transAxes)
        if i==len(oceans)-1:
            ax_bot.set_xlabel('time [model years]')
        else:
            ax_bot.set_xticks([])
            
            
cax = fig.add_axes([0.1, 0.04, 0.88, 0.02])
fig.colorbar(im, cax=cax, orientation='horizontal', label='OHC anomaly [ZJ/m]', extend='both')
# plt.savefig(f'{path_results}/paper/OHC_vertical_Hovmoeller_0-6km_ctrl_lpd_qd')


In [ ]:
x

# Table 2: variances for different bandpass filters

How much area/volume do the major ocean basins represent?

In [ ]:
from xr_DataArrays import xr_DZ, xr_AREA
from paths import file_RMASK_ocn
from regions import regions_dict
DZT = xr_DZ('ocn')
AREA = xr_AREA('ocn')
MASK = xr.open_dataarray(file_RMASK_ocn)

In [ ]:
AREA

In [ ]:
total_area = AREA.where(MASK>0).sum(dim=['nlat','nlon'],skipna=True).values
total_volume = (DZT*AREA.where(MASK>0)).sum(dim=['z_t','nlat','nlon'],skipna=True).values
print(f'total area {total_area} volume {total_volume}')
for i in [1,2,6]:
    area = AREA.where(MASK==i).sum(dim=['nlat','nlon'],skipna=True).values
    volume = (DZT*AREA.where(MASK==i)).sum(dim=['z_t','nlat','nlon'],skipna=True).values
    print(f'{regions_dict[i]:15}: area {area/total_area*100:2.0f}%; volume {volume/total_volume*100:2.0f}%')
    

In [ ]:
25.8+38.1+17.7

In [ ]:
26.8+41.2+17.8